# Basic informatic functions

This notebook showcases some basic functions that demonstrate some of the analytic insights available using the legal-citation-parser tool. These simple proof-of-concept functions are intended to demonstrate that the tool is capable of assisting with basic legal informatics.

## Imports

The basic informatic functions are built around the ``legal-citation-parser`` package. For simple data analysis, I use the Counter class from ``collections``

In [ ]:
import matplotlib.pyplot as plt

from collections import Counter
from legal_citation_parser import create_citation, parse_citation

## Constants

For now, the only constants this program needs are the lengthened province & territory names that allow the program to display more readable output.

In [ ]:
# Constants file for output strings

ABBREVIATION_MAP = {
    "bc": "British Columbia",
    "ab": "Alberta",
    "sk": "Saskatchewan",
    "mb": "Manitoba",
    "on": "Ontario",
    "qc": "Quebec",
    "nb": "New Brunswick",
    "ns": "Nova Scotia",
    "pe": "Prince Edward Island",
    "nl": "Newfoundland and Labrador",
    "yt": "Yukon",
    "nt": "Northwest Territories",
    "nu": "Nunavut",
    "ca": "Canada",
}


In [ ]:
def validate_kwargs(citation: str, cited: bool, citing: bool) -> list:
    """
    DRY function used to validate the keyword arguments passed to the function. 
    Future analytics may want to use both cited and citing kwargs, but that's not
    the case for these basic ones.

    Args:
        citation (str): The citation string to be parsed.
        cited (bool): Whether the function is to return cited cases.
        citing (bool): Whether the function is to return citing cases.

    Returns:
        case_list (list): A list of cases that are either cited or citing.
    """

    if cited and not citing:
        case = create_citation(citation, cited=True)
        case_info = case.parse()
        case_list = case_info["cited_cases"]['citedCases']

        return case_list
    
    elif citing and not cited:
        case = create_citation(citation, citing=True)
        case_info = case.parse()
        case_list = case_info["citing_cases"]['citingCases']

        return case_list
    
    else:
        
        return {"error": "Function must operate on either the cited_cases or citing_cases list. Run jurisdiction(citation, cited=True) or (... citing=True)."}

## Jurisdiction/court level analysis using cited_cases

The first function, ``jurisdiction_analysis``, takes a list of cases and returns a dictionary with the number of cases from each jurisdiction. It also returns the number of cases from each respective court level, as well as the number of cases from particular court levels in particular jurisdictions. These figures can show which jurisdictions and court levels are most frequently cited, and to determine whether there are regional differences in citation patterns.

In [ ]:
def jurisdiction(citation: str, cited=False, citing=False) -> dict:
    """
    Function to return the jurisdiction, court level, and court name of a citation.

    Args:
        citation (str): The citation string to be parsed.
        cited (bool): Whether the function is to return cited cases.
        citing (bool): Whether the function is to return citing cases.

    Returns:
        jurisdiction_counts (dict): A dictionary of jurisdiction counts.
        level_counts (dict): A dictionary of court level counts.
        court_counts (dict): A dictionary of court counts.
    """
 
    case_list = validate_kwargs(citation, cited, citing)
    parsed_case_list = []

    for case in case_list:
        case_info = parse_citation(case["title"] + ", " + case["citation"])
        parsed_case_list.append(case_info)

    # Replace abbreviations with full names
    for case in parsed_case_list:
        if case["jurisdiction"] in ABBREVIATION_MAP:
            case["jurisdiction"] = ABBREVIATION_MAP[case["jurisdiction"]]

    jurisdictions = [case["jurisdiction"] for case in parsed_case_list]
    jurisdiction_counts = Counter(jurisdictions)

    levels = [case["court_level"] for case in parsed_case_list]
    level_counts = Counter(levels)

    courts = [case["jurisdiction"] + " " + case["court_level"] for case in parsed_case_list]
    court_counts = Counter(courts)

    return jurisdiction_counts, level_counts, court_counts

def jurisdiction_analysis(citation: str):
    """
    Function to print out the jurisdiction analysis of a citation.
    """
    
    analysis = jurisdiction(citation, cited=True)

    print("\nJusrisdiction analysis\n==========")
    for item, count in analysis[0].items():
        print(f"{item}: {count}")

    print("\n\nLevel analysis\n==========")
    for item, count in analysis[1].items():
        print(f"{item}: {count}")

    print("\n\nCourt analysis\n==========")
    for item, count in analysis[2].items():
        print(f"{item}: {count}")

### User input

Run the cell below to input a citation. Note: this program requires a CanLII API key to run, and will prompt the user for one if one is not stored as an environment variable.

In [ ]:
citation = input("Enter a CanLII citation: ")
jurisdiction_analysis(citation)

## Citation frequency analysis using citing_cases

The second function, ``citation_frequency``, takes a list of cases and returns a dictionary with the number of times each case is cited. This can be useful for determining whether citations for a particular case have increased, decreased, or remained steady over time. Sharp increases in citations may indicate that a case has become more relevant, while sharp decreases may indicate that a case has become less relevant or has been overturned/overruled.

In [ ]:
def citation_frequency(citation, cited=False, citing=False) -> dict:
    """
    Function to count the frequency of cases by year.

    Args:
        citation (str): The citation string to be parsed.
        cited (bool): Whether the function is to return cited cases.
        citing (bool): Whether the function is to return citing cases.

    Returns:
        year_counts (dict): A dictionary with the year as the key and the frequency of cases as the value.
    """

    case_list = validate_kwargs(citation, cited, citing)
    parsed_case_list = []

    # Count the frequency of each case by year, using the year as the key
    # Note that each case has a "year" attribute
    
    for case in case_list:
        case_info = parse_citation(case["title"] + ", " + case["citation"])
        parsed_case_list.append(case_info["year"])

    year_counts = Counter(parsed_case_list)

    return year_counts

def citation_frequency_analysis(citation: str):
    """
    Plots a bar graph of the citation frequency analysis for a given citation.

    Args:
        citation (str): The citation string to be parsed

    Returns:
        None
    """

    citation_frequency_analysis = citation_frequency(citation, citing=True)
    years = sorted(citation_frequency_analysis.keys())
    frequencies = [citation_frequency_analysis[year] for year in years]

    # Get the case name
    case = create_citation(citation)
    case_name = case.style_of_cause + ", " + case.atomic_citation

    # Create a bar plot
    plt.figure(figsize=(10, 6))
    plt.bar(years, frequencies, color='orange')
    plt.title(f'Citation Frequency Analysis for {case_name}')
    plt.xlabel('Year')
    plt.ylabel('Frequency')

    # Show the plot
    plt.show()


### User input

Run the cell below to input a citation. Note: this program requires a CanLII API key to run, and will prompt the user for one if one is not stored as an environment variable.

In [ ]:
citation = input("Enter a CanLII citation: ")
citation_frequency_analysis(citation)